In [2]:
import pandas as pd
import numpy as np
import spacy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import nltk

from sklearn.preprocessing import LabelEncoder
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.model_selection import train_test_split, KFold

In [3]:
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

print(data_train.info())
print("Test info : \n ", data_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3235 entries, 0 to 3234
Data columns (total 6 columns):
id                 3235 non-null float64
original_text      3235 non-null object
lang               3231 non-null object
retweet_count      3235 non-null int64
original_author    3235 non-null object
sentiment_class    3235 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 151.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1387 entries, 0 to 1386
Data columns (total 5 columns):
id                 1387 non-null float64
original_text      1387 non-null object
lang               1387 non-null object
retweet_count      1387 non-null int64
original_author    1387 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 54.3+ KB
Test info : 
  None


In [4]:
for i in range(len(data_train.id)):
    if(type(data_train.retweet_count[i])==float):
        print(i)
        data_train.retweet_count[i] = "0"

for i in range(len(data_train.id)):
    if(data_train.retweet_count[i].isnumeric()):
        print(i)
        data_train.retweet_count[i] = int(data_train.retweet_count[i])
    else:
        if(type(data_train.original_author[i]).isnumeric()):
            data_train.retweet_count[i] = data_train.original_aurthor[i]
        else:
            data_train.retweet_count[i] = 0
            

        
        

AttributeError: 'numpy.int64' object has no attribute 'isnumeric'

In [ ]:
data_train.info()

data_train.retweet_count.astype('int16')
print((data_train.retweet_count).dtype)
data_train.info()

In [5]:
for i in data_train.retweet_count:
    print(i)

0
1
0
0
0
0
0
0
0
0
0
3
1
0
1
0
0
0
0
11
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
12
0
0
0
2
0
0
0
0
1
1
0
0
0
0
0
0
1
0
0
3
0
0
0
0
0
1
0
1
0
0
1
0
0
0
0
0
4
0
0
0
0
0
2
0
0
0
0
0
1
0
0
1
0
0
1
0
3
0
1
1
0
0
0
0
0
2
0
0
0
0
0
5
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
24
0
0
1
0
0
0
1
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0
7
0
0
1
0
0
3
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
1
0
0
0
0
20
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
24
0
1
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
1
0
0
2
0
0
0
0
2
0
0
0
3
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
1
1
0
1
0
0
0
0
0
2
0
0
0
0
0
1
0
0
0
0
0
5
0
0
0
0
0
0
2
0
0
0
0
2
0
2
5
0
0
0
0
11
0
0
0
0
0
0
0
0
2
0
1
16
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
2
0
0
0
0
4
0
2
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
32
0
0
0
1
0
0


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [7]:
X = pd.DataFrame(columns = ['original_text', "retweet_count"])
X['original_text'] = data_train['original_text'].copy
X["retweet_count"] = data_train["retweet_count"].copy
y = data_train['sentiment_class']
nlp = sp.load("en_core_web_sm")

In [9]:
import re
def processing(txt):
    txt.lower()
    txt = re.sub(r"(pic.twitter.com.*)", "", txt)
    txt = re.sub(r"(https:.*)", "", txt)
    txt = re.sub("(#+)", "", txt)
    txt = re.sub("(@+)", "", txt)
#     txt = re.sub("($+)", "", txt)
    txt = re.sub("(%+)", "", txt)
    txt = re.sub("(&+)", "", txt)
    txt = re.sub("\s+", " ", txt)
#     txt = re.sub(" ", " ", txt)
#     print(txt)
    obj = nlp(txt)
    txt1 = [wd.text for wd in obj if wd.is_stop != True and wd.pos_ != 'PUNCT' ]
    txt1 = " ".join(txt1)
    txt1 = re.sub("\s+", " ", txt1)
#     print(txt1)
    return(txt1)
    
txt = data_train.original_text[3234]
print(txt)
processing(txt)

Wishing you all a safe & happy Mothers Day #mothersday #reedham #relax @Reedham ferry Inn. https://www. instagram.com/p/B-Coi97jcDG/ ?igshid=1wk66ouvsqkt3 …


'Wishing safe happy Mothers Day mothersday reedham relax Reedham ferry Inn'

In [10]:
data_train.original_text = data_train.original_text.apply(processing)
data_train.drop(["id", "lang", "original_author", "sentiment_class"], axis =1, inplace=True)

In [35]:
print(type(data_train.original_text[0]))

<class 'str'>


# Dont Go above this

In [29]:
X_train, X_test, y_train, y_test = train_test_split(data_train, y, test_size = 0.3, random_state= 10)
print(X_train, X_test.shape)
print(y_train.shape, y_test)

                                          original_text  retweet_count
1586  support local businesses difficult time ordere...              0
2646  Hope okay inspiration Liam Honestly men like i...              0
1266  Happy mothers day wonderful mums Today gave mu...              1
1801  mothers grandmothers Dad mothers dog mothers p...              0
1010  Happy Mother Day celebration honoring mother f...              0
...                                                 ...            ...
3195  Mum special person world love heart Happy Moth...              0
1344  Happy Mother Day PAN UK mums Nia Sian Ellie Ro...              1
527   Happy MothersDay Hubby serving mom baked pasta...              1
3197  Isan Elba celebrates Happy Mothers ’ Day Mom H...              0
1289  Happy Mother Day MothersDay Horror Jason Frida...              4

[2264 rows x 2 columns] (971, 2)
(2264,) 1246    1
820     0
2108    1
2641   -1
2177    1
       ..
331     1
2145    1
3038    1
2091   -1
464   

In [30]:
tf = TfidfVectorizer()

In [31]:
X_train.original_text

1586    support local businesses difficult time ordere...
2646    Hope okay inspiration Liam Honestly men like i...
1266    Happy mothers day wonderful mums Today gave mu...
1801    mothers grandmothers Dad mothers dog mothers p...
1010    Happy Mother Day celebration honoring mother f...
                              ...                        
3195    Mum special person world love heart Happy Moth...
1344    Happy Mother Day PAN UK mums Nia Sian Ellie Ro...
527     Happy MothersDay Hubby serving mom baked pasta...
3197    Isan Elba celebrates Happy Mothers ’ Day Mom H...
1289    Happy Mother Day MothersDay Horror Jason Frida...
Name: original_text, Length: 2264, dtype: object

In [32]:
a = tf.fit_transform(X_train.original_text)
X = tf.transform(X_train.original_text)
X

<2264x8213 sparse matrix of type '<class 'numpy.float64'>'
	with 34980 stored elements in Compressed Sparse Row format>

In [33]:
type(X)

scipy.sparse.csr.csr_matrix

In [16]:
tf.get_feature_names()

['01506',
 '0151',
 '0161',
 '02073833333',
 '03',
 '04',
 '0800',
 '08130397997',
 '08th',
 '10',
 '100',
 '100daysofwalking',
 '105',
 '1059',
 '10th',
 '11',
 '12',
 '1241782300796424193',
 '12s',
 '13',
 '1430',
 '15',
 '1584453304sr',
 '15yrs',
 '16',
 '1625',
 '1667',
 '18',
 '1800',
 '18th',
 '19',
 '1919',
 '1926',
 '1939',
 '1956',
 '1971',
 '1980',
 '1983',
 '1992',
 '1harrisonrose',
 '1lovetoallmothers',
 '1of',
 '1s',
 '1st',
 '1stmothersday',
 '1年間の感謝を込めて',
 '20',
 '200',
 '2005',
 '2010',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2020mothersday',
 '2020年母の日は5月10日',
 '2021',
 '2023',
 '20snana',
 '21',
 '21lva',
 '21st',
 '22',
 '22nd',
 '2348034127257',
 '24',
 '25',
 '25th',
 '26',
 '26am',
 '27',
 '28',
 '28ish',
 '29',
 '2all',
 '2day',
 '2metres',
 '2nd',
 '2niteflavour',
 '2u6pm9j',
 '30',
 '3000',
 '30pm',
 '31',
 '365',
 '36holes',
 '3days',
 '3k97fm',
 '3k97fn',
 '3metres',
 '3positives',
 '3rd',
 '3yo',
 '40',
 '41p4',
 '42',
 '42weekspregn

In [23]:
cv = CountVectorizer()
a = cv.fit_transform(X_train.original_text)
X_train.original_text = cv.transform(X_train.original_text)

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [24]:
X_train.original_text.values

array([<2264x8213 sparse matrix of type '<class 'numpy.int64'>'
	with 34980 stored elements in Compressed Sparse Row format>,
       <2264x8213 sparse matrix of type '<class 'numpy.int64'>'
	with 34980 stored elements in Compressed Sparse Row format>,
       <2264x8213 sparse matrix of type '<class 'numpy.int64'>'
	with 34980 stored elements in Compressed Sparse Row format>,
       ...,
       <2264x8213 sparse matrix of type '<class 'numpy.int64'>'
	with 34980 stored elements in Compressed Sparse Row format>,
       <2264x8213 sparse matrix of type '<class 'numpy.int64'>'
	with 34980 stored elements in Compressed Sparse Row format>,
       <2264x8213 sparse matrix of type '<class 'numpy.int64'>'
	with 34980 stored elements in Compressed Sparse Row format>],
      dtype=object)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [26]:
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

forest = RandomForestClassifier(random_state = 1)

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(forest, hyperF, cv = 3, verbose = 1, n_jobs = -1)
gridF.fit(X, y_train)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   49.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 33.7min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 42.5min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=1,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max

In [27]:
from sklearn.metrics import f1_score, confusion_matrix
XX = tf.transform(X_test.original_text)
y_pred = gridF.predict(XX)
f1_score(y_pred, y_test, average='weighted')

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.693135935397039

In [259]:
confusion_matrix(y_pred, y_test)

array([[  0,   0,   0],
       [226, 515, 230],
       [  0,   0,   0]], dtype=int64)

In [270]:
rf = RandomForestClassifier()
rf.fit(X, y_train)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion="gini", max_depth=None, max_features='auto', max_leaf_nodes=None,min_impurity_split=1e-07, min_samples_leaf=1,min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1, oob_score=False, random_state=None, verbose=0, warm_start=False)
y_pred = rf.predict(XX)
print(f1_score(y_pred, y_test, average='weighted'))
confusion_matrix(y_test, y_pred)

C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5401976369001951


array([[ 29, 184,  13],
       [ 71, 411,  33],
       [ 26, 184,  20]], dtype=int64)

In [266]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, confusion_matrix

nb = GaussianNB()
nb.fit(X.toarray(), y_train)
y_pred = nb.predict(XX.toarray())
f1_score(y_pred, y_test, average='weighted')

0.35724037742491127

In [268]:
confusion_matrix(y_test, y_pred)

array([[ 89,  92,  45],
       [189, 215, 111],
       [ 75, 105,  50]], dtype=int64)

In [77]:
from scipy.stats import mode
s = "1 2 2 2 2 "
a = s.split(" ")
print(a)
a = [int(k) for k in a]
a = pd.DataFrame(a)


x = a.mode().sample(frac=1)
print(x.values)

['1', '2', '2', '2', '2', '']


ValueError: invalid literal for int() with base 10: ''